In [940]:
import numpy as np
from scipy.stats import beta

# parameters
sigma = 5               # elasticity of substitution between critical goods
theta = 0.1             # between outside and critical good
kappa = 0.5             # fixed entry cost
z_h = {"A": 0.0, "B": 0.0} # production subsidies for countries A and B
e_h = {"A": 0.0, "B": 0.0}  # entry subsidy for countries A and B
tau = 0.1                   # generic trade cost

imtax_Fij = {"A": {"A": 0.00, "B": 0.00, "C": 0.00}, # import tariff on critical goods from i to j
             "B": {"A": 0.00, "B": 0.00, "C": 0.00}}

extax_Fij = {"A": {"A": 0.00, "B": 0.00}, # export tax on critical goods from i to j
             "B": {"A": 0.00, "B": 0.00}}

imtax_Ihi = {"A": {"A": 0.00, "B": 0.00}, # import tariff on inputs from h to i
             "B": {"A": 0.00, "B": 0.00}}

extax_Ihi = {"A": {"A": 0.00, "B": 0.00}, # export tax on inputs from h to i
             "B": {"A": 0.00, "B": 0.00}}

alpha_c = 19    # alpha parameter for country risk, e.g. 5.67 (~85%), 9 (90%), 19 (95%), 99 (99%)
beta_c = 1      # beta parameter (fix to 1)
alpha_i = 19    # alpha parameter for idiosyncratic risk
beta_i = 1      # beta parameter (fix to 1)

cvar_alpha = 0.95  # confidence level for CVaR

In [962]:
# run model
# trade costs for critical goods from A to (A, B)
t_Fij = {
    i: {
        j: 1 if i == j else 1 + imtax_Fij.get(i, {}).get(j, 0) + extax_Fij.get(i, {}).get(j, 0) + tau
        for j in ("A", "B", "C") if j in imtax_Fij.get(i, {})
    }
    for i in ("A", "B", "C") if i in imtax_Fij
}

# trade costs for critical goods from A to (A, B)
t_Ihi = {
    h: {
        i: 1 if h == i else 1 + imtax_Ihi.get(h, {}).get(i, 0) + extax_Ihi.get(h, {}).get(i, 0) + tau
        for i in ("A", "B", "C") if i in imtax_Ihi.get(h, {})
    }
    for h in ("A", "B") if h in imtax_Ihi
}

# expected survival probability
expected_phi = (alpha_c/(alpha_c + beta_c))*(alpha_i/(alpha_i + beta_i))

# calculate input prices
p_Ih = {
    "A": (sigma / (sigma - 1)) * (1 / expected_phi - z_h["A"]),
    "B": (sigma / (sigma - 1)) * (1 / expected_phi - z_h["B"]),
}

# calculate price of final goods (p_Fi) = input price index (bar_P_Ii)
p_Fi = {i: np.sum([(p_Ih[h] * t_Ihi[h][i]) ** (1 - sigma) for h in ("A", "B")]) ** (1 / (1 - sigma)) for i in ("A", "B")}

# calculate price index for final goods consumed in country j
bar_P_Fj = {j: np.sum([(p_Fi[i] * t_Fij[i][j]) ** (1 - sigma) for i in ("A", "B")]) ** (1 / (1 - sigma)) for j in ("A", "B", "C")}

# expected demand for final goods
expected_x_Fij = {
    i: {j: (p_Fi[i] * t_Fij[i][j]) ** (-sigma) * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta)) for j in ("A", "B", "C")}
    for i in ("A", "B")
}

# expected demand for inputs
expected_X_Ihi = {
    h: {
        i: (p_Ih[h] * t_Ihi[h][i]) ** -sigma
        * sum(
            t_Fij[i][j] ** -sigma
            * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta))
            for j in ("A", "B", "C")
        )
        for i in ("A", "B")
    }
    for h in ("A", "B")
}

# n_h for each h
n_h = {
    h: sum(expected_x_Fij[i][h] for i in ("A", "B"))
    / ((sigma - 1) * (1 - e_h[h]) * kappa)
    * ((1 / expected_phi) - z_h[h])
    for h in ("A", "B")
}

# expected survivors
ns_h = {
    h: n_h[i] * expected_phi for i in ("A", "B")
    for h in ("A", "B")
}

# expected tariff revenue
tariff_revenue = {
    j: sum(
        imtax_Fij.get(i, {}).get(j, 0) * expected_x_Fij.get(i, {}).get(j, 0)
        for i in ("A", "B", "C") if h != j
    ) + (sum(
        imtax_Ihi.get(i, {}).get(j, 0) * expected_X_Ihi.get(i, {}).get(j, 0)
        for i in ("A", "B") if h != j
    ) if j in ("A", "B") else 0)
    + sum(
        extax_Fij.get(j, {}).get(i, 0) * expected_x_Fij.get(j, {}).get(i, 0)
        for i in ("A", "B") if j != i
    )
    + (sum(
        extax_Ihi.get(j, {}).get(i, 0) * expected_X_Ihi.get(j, {}).get(i, 0)
        for i in ("A", "B") if j != i
    ) if h in ("A", "B") else 0)
    for j in ("A", "B", "C")
}

# expected production/entry subsidy cost
subsidy_cost = {
    h: ns_h.get(h, 0) * e_h.get(h, 0) + z_h.get(h, 0) * sum(
        expected_X_Ihi.get(h, {}).get(i, 0) for i in ("A", "B")
    )
    for h in ("A", "B")
}

# outside good consumption
x_0j = {
    j: 1
    + tariff_revenue.get(j, 0)
    - subsidy_cost.get(j, 0)
    - bar_P_Fj.get(j, 0) ** (-theta / (1 - theta))
    for j in ("A", "B", "C")
}

# monte carlo setup
num_simulations = 30000
max_iterations = 1000
iteration = 0
results = []
cvar_results = []
tolerance = 0.01  # relative error tolerance
z_alpha = 1.96    # critical z-value for 95% confidence

# run the loop until convergence or max iterations
while iteration < max_iterations:
    iteration += 1   
    
    # Generate risk factors
    D_h_A = 1 - beta.rvs(alpha_c, beta_c, size=num_simulations)
    D_h_B = 1 - beta.rvs(alpha_c, beta_c, size=num_simulations)
    fail_all_A = np.random.rand(num_simulations) < D_h_A
    fail_all_B = np.random.rand(num_simulations) < D_h_B
    phi_A = np.where(fail_all_A, 0, beta.rvs(alpha_i, beta_i, size=num_simulations))
    phi_B = np.where(fail_all_B, 0, beta.rvs(alpha_i, beta_i, size=num_simulations))
    
    # Compute rho_i directly for all samples
    rho_i = {
        i: np.sum(
            (np.stack([phi_A, phi_B], axis=0) / expected_phi) ** ((sigma - 1) / sigma)
            * np.array([p_Ih["A"], p_Ih["B"]])[:, None] ** (1 - sigma)
            * np.array([t_Ihi["A"][i], t_Ihi["B"][i]])[:, None] ** (1 - sigma),
            axis=0,
        ) ** (sigma / (sigma - 1))
        * p_Fi[i] ** sigma
        for i in ("A", "B")
    }
    
    # compute utility U_j
    sum_term = {
        j: 
        (((rho_i["A"] * expected_x_Fij["A"][j]) ** ((sigma - 1) / sigma) + 
          (rho_i["B"] * expected_x_Fij["B"][j]) ** ((sigma - 1) / sigma)))
        for j in ("A", "B", "C")
    }
    
    U_j = {
        j: x_0j.get(j, 0) +
           (1 / theta) * (sum_term.get(j, 0) ** (theta * sigma / (sigma - 1)))
        for j in ("A", "B")
    }
    
    # Calculate VaR as a dictionary
    var_threshold = {
        j: np.percentile(U_j[j], (1 - cvar_alpha) * 100) for j in U_j.keys()
    }
    
    # Calculate CVaR as a dictionary
    cvar = {
        j: np.mean([u for u in U_j[j] if u <= var_threshold[j]]) for j in U_j.keys()
    }
    
    # Append results
    cvar_results.append(cvar)
    results.append({j: np.mean(U_j[j]) for j in U_j.keys()})

    # Extract utility and CVaR results for country "A"
    cvar_results_A = [cvar["A"] for cvar in cvar_results]
    results_A = [result["A"] for result in results]
    
    # Calculate standard errors for country "A"
    cvar_sem_A = np.std(cvar_results_A, ddof=1) / np.sqrt(len(cvar_results_A))
    mean_sem_A = np.std(results_A, ddof=1) / np.sqrt(len(results_A))
    
    # Calculate confidence interval widths for country "A"
    mean_ci_width_A = 2 * z_alpha * mean_sem_A
    cvar_ci_width_A = 2 * z_alpha * cvar_sem_A
    
    # Check convergence for country "A"
    mean_converged_A = (mean_ci_width_A / np.abs(np.mean(results_A))) < tolerance
    cvar_converged_A = (cvar_ci_width_A / np.abs(np.mean(cvar_results_A))) < tolerance
    
    if mean_converged_A and cvar_converged_A:
        print(f"Converged after {iteration} iterations.")
        break
else:
    print("Reached maximum iterations without convergence.")

# Extract utility and CVaR results for countries A and B
results_A = [result["A"] for result in results]
results_B = [result["B"] for result in results]
cvar_results_A = [cvar["A"] for cvar in cvar_results]
cvar_results_B = [cvar["B"] for cvar in cvar_results]

# Output final results
print(f"Overall mean utility (A): {np.mean(results_A):.3f}")
print(f"Overall mean utility (B): {np.mean(results_B):.3f}")
print(f"Mean CVaR (at α = {cvar_alpha}) (A): {np.mean(cvar_results_A):.3f}")
print(f"Mean CVaR (at α = {cvar_alpha}) (B): {np.mean(cvar_results_B):.3f}")

Converged after 145 iterations.
Overall mean utility (A): 9.497
Overall mean utility (B): 9.496
Mean CVaR (at α = 0.95) (A): 4.904
Mean CVaR (at α = 0.95) (B): 4.903
